In [0]:
# Copyright 2019 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#Predicting Party Classification with BERT on TF Hub

Modified from: https://colab.research.google.com/github/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb

In [0]:
!pip3 install --quiet "tensorflow>=1.7"

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import datetime
from datetime import timedelta

W0608 21:38:57.482674 140482351376256 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


Installing BERT's python package.

In [3]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 3.4MB/s 


In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

Storing model to GCP bucket.

In [5]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'TEST_3'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = True #@param {type:"boolean"}
BUCKET = 'text-classification-221' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: gs://text-classification-221/TEST_3 *****


#Data

Uploading the files with all of the tweets.

In [0]:
from google.colab import files

In [8]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [0]:
uploaded = files.upload()

Saving new_data_cleaned_train.csv to new_data_cleaned_train.csv
Saving new_data_cleaned_test.csv to new_data_cleaned_test.csv
Saving new_data_cleaned_rep.csv to new_data_cleaned_rep.csv
Saving new_data_cleaned_dem.csv to new_data_cleaned_dem.csv


In [0]:
allData = pd.read_csv('party_data2-label_and_text.csv', header=None, names=['text', 'classification'])
train = pd.read_csv('train.csv', header=None, names=['text', 'classification'])
test = pd.read_csv('test.csv', header=None, names=['text', 'classification'])

Removes the duplicate tweets from the data set and shuffles the data.

In [10]:
allData.drop_duplicates(subset ="text", inplace = True)

115190
110180


In [12]:
allData = allData.sample(len(allData))
train = train.sample(len(train))
test = test.sample(len(test))
print (len(allData))
print (len(train))
print (len(test))

74324
35856


In [0]:
print ('Training data breakdown')
train.groupby('classification').size()

Training data breakdown


classification
0    36462
1    37862
dtype: int64

In [0]:
print ('Testing data breakdown')
test.groupby('classification').size()

Testing data breakdown


classification
0    17737
1    18119
dtype: int64

In [0]:
print ('All data breakdown')
allData.groupby('classification').size()

All data breakdown


classification
0    54199
1    55981
dtype: int64

The input data is the 'text' column (with the tweets) and the label is the 'classification' column (0 for Democratic and 1 for Republican).

In [0]:
DATA_COLUMN = 'text'
LABEL_COLUMN = 'classification'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

#Data Preprocessing
(From the original colab file)
We'll need to transform our data into a format BERT understands. This involves two steps. First, we create  `InputExample`'s using the constructor provided in the BERT library.

- `text_a` is the text we want to classify, which in this case, is the `Request` field in our Dataframe. 
- `text_b` is used if we're training a model to understand the relationship between sentences (i.e. is `text_b` a translation of `text_a`? Is `text_b` an answer to the question asked by `text_a`?). This doesn't apply to our task, so we can leave `text_b` blank.
- `label` is the label for our example, i.e. True, False

In [0]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

Next, we need to preprocess our data so that it matches the data BERT was trained on. For this, we'll need to do a couple of things (but don't worry--this is also included in the Python library):


1. Lowercase our text (if we're using a BERT lowercase model)
2. Tokenize it (i.e. "sally says hi" -> ["sally", "says", "hi"])
3. Break words into WordPieces (i.e. "calling" -> ["call", "##ing"])
4. Map our words to indexes using a vocab file that BERT provides
5. Add special "CLS" and "SEP" tokens (see the [readme](https://github.com/google-research/bert))
6. Append "index" and "segment" tokens to each input (see the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf))

Happily, we don't have to worry about most of these details.




To start, we'll need to load a vocabulary file and lowercasing information directly from the BERT tf hub module:

In [16]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

Instructions for updating:
Colocations handled automatically by placer.


W0608 21:41:31.500290 140482351376256 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0608 21:41:33.546241 140482351376256 saver.py:1483] Saver not created because there are no variables in the graph to restore


Great--we just learned that the BERT model we're using expects lowercase data (that's what stored in tokenization_info["do_lower_case"]) and we also loaded BERT's vocab file. We also created a tokenizer, which breaks words into word pieces:

Using our tokenizer, we'll call `run_classifier.convert_examples_to_features` on our InputExamples to convert them into features BERT understands.

In [17]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 74324


I0608 21:41:36.279325 140482351376256 run_classifier.py:774] Writing example 0 of 74324


INFO:tensorflow:*** Example ***


I0608 21:41:36.284995 140482351376256 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

I0608 21:42:28.263969 140482351376256 run_classifier.py:774] Writing example 10000 of 35856


INFO:tensorflow:Writing example 20000 of 35856


I0608 21:42:34.166156 140482351376256 run_classifier.py:774] Writing example 20000 of 35856


INFO:tensorflow:Writing example 30000 of 35856


I0608 21:42:40.084116 140482351376256 run_classifier.py:774] Writing example 30000 of 35856


#Creating a model

(From the original colab file) Now that we've prepared our data, let's focus on building a model. `create_model` does just this below. First, it loads the BERT tf hub module again (this time to extract the computation graph). Next, it creates a single new layer that will be trained to adapt BERT to our sentiment task (i.e. classifying whether a movie review is positive or negative). This strategy of using a mostly trained model is called [fine-tuning](http://wiki.fast.ai/index.php/Fine_tuning).

In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


Next we'll wrap our model function in a `model_fn_builder` function that adapts our model to work for training, evaluation, and prediction.

In [0]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [0]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [0]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [23]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_model_dir': 'gs://text-classification-221/TEST_3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc423f17198>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0608 21:42:43.680194 140482351376256 estimator.py:201] Using config: {'_model_dir': 'gs://text-classification-221/TEST_3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc423f17198>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Next we create an input builder function that takes our training feature set (`train_features`) and produces a generator. This is a pretty standard design pattern for working with Tensorflow [Estimators](https://www.tensorflow.org/guide/estimators).

In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

Now we train our model! For me, using a Colab notebook running on Google's GPUs, my training time was about 14 minutes.

In [0]:
print('--- Beginning Training! ---')
current_time = datetime.datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("--- Training time took {} ---".format(datetime.datetime.now() - current_time))

--- Beginning Training! ---
INFO:tensorflow:Calling model_fn.


I0607 16:40:11.048156 139888344217472 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0607 16:40:15.029648 139888344217472 saver.py:1483] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

INFO:tensorflow:Saving checkpoints for 6967 into gs://text-classification-221/TEST_3/model.ckpt.


I0607 18:31:34.841876 139888344217472 basic_session_run_hooks.py:594] Saving checkpoints for 6967 into gs://text-classification-221/TEST_3/model.ckpt.


INFO:tensorflow:Loss for final step: 0.036180634.


I0607 18:32:14.685513 139888344217472 estimator.py:359] Loss for final step: 0.036180634.


--- Training time took 1:52:43.317466 ---


Now let's use our test data to see how well our model did:

In [0]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [0]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


I0607 18:32:51.437971 139888344217472 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0607 18:32:55.417135 139888344217472 saver.py:1483] Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 6967: auc = 0.9003143, eval_accuracy = 0.9003514, f1_score = 0.901638, false_negatives = 1743.0, false_positives = 1830.0, global_step = 6967, loss = 0.49645168, precision = 0.8994837, recall = 0.90380263, true_negatives = 15907.0, true_positives = 16376.0


I0607 18:38:52.370143 139888344217472 estimator.py:1979] Saving dict for global step 6967: auc = 0.9003143, eval_accuracy = 0.9003514, f1_score = 0.901638, false_negatives = 1743.0, false_positives = 1830.0, global_step = 6967, loss = 0.49645168, precision = 0.8994837, recall = 0.90380263, true_negatives = 15907.0, true_positives = 16376.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 6967: gs://text-classification-221/TEST_3/model.ckpt-6967


I0607 18:38:59.124486 139888344217472 estimator.py:2039] Saving 'checkpoint_path' summary for global step 6967: gs://text-classification-221/TEST_3/model.ckpt-6967


{'auc': 0.9003143,
 'eval_accuracy': 0.9003514,
 'f1_score': 0.901638,
 'false_negatives': 1743.0,
 'false_positives': 1830.0,
 'global_step': 6967,
 'loss': 0.49645168,
 'precision': 0.8994837,
 'recall': 0.90380263,
 'true_negatives': 15907.0,
 'true_positives': 16376.0}

# Processing Republican and Democratic Tweets Keywords

Converting each party's tweets into data frames and then turning the tweets into lists.

In [25]:
!pip install rake-nltk

  Stored in directory: /root/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk


In [26]:
import nltk
nltk.download('punkt')
import math
import numpy as np
from rake_nltk import Rake
from nltk import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
repParty = allData.loc[allData['classification'] == 1]
demParty = allData.loc[allData['classification'] == 0]
#repParty = pd.read_csv('new_data_cleaned_rep.csv', header=None, names=['text', 'classification'])
#demParty = pd.read_csv('new_data_cleaned_dem.csv', header=None, names=['text', 'classification'])

In [0]:
def extractTweets(data, test):
  originalSentences = []
  for index, row in data.iterrows():
    if test:
      originalSentences.append((row['text'].lower(), row['classification']))
    else:
      originalSentences.append(row['text'])
  return originalSentences

In [29]:
originalRep = extractTweets(repParty, False)
originalDem = extractTweets(demParty, False)
print (originalRep[:5])
print (originalDem[:5])

['with florida new governor taking office some parents of students killed in the marjory stoneman douglas high massacre are pushing for the removal of broward sheriff scott israel', 'common sense any feeling person knows this is not true is it really ok to kill a 7-pound baby that is just not born yet is not there a point where that life matters too this is why we need events like', 'want to know what the american health care act will mean for your family click play to find out', 'instead we should promote innovation to reduce carbon dioxide emissions promising new technologies like advanced nuclear power carbon capture and carbon utilization hold the key to significant emissions reductions we can lower emissions without crashing the economy', 'i am leading senate_go ps in filing an amicus brief to scotus in support of a challenge to potus executive actions on immigration']
['thank you sen. lisamurkowski for standing with us and voting to protect our health rights and freedoms when it 

Preprocessing: extracting all of the keywords from the Republican and Democratic tweets in preparation for later comparison with the input.

In [0]:
def getBaseKeywords(originalData):
  originalKeywords = []
  r = Rake()
  
  for tweet in originalData:
    r.extract_keywords_from_text(tweet)
    originalKeywords.append(r.get_ranked_phrases())
  
  return originalKeywords

In [31]:
allRepKeywords = getBaseKeywords(originalRep)
allDemKeywords = getBaseKeywords(originalDem)
print (allRepKeywords[:5])
print (allDemKeywords[:5])

[['marjory stoneman douglas high massacre', 'florida new governor taking office', 'broward sheriff scott israel', 'students killed', 'removal', 'pushing', 'parents'], ['need events like', 'feeling person knows', 'really ok', 'pound baby', 'life matters', 'common sense', 'born yet', 'true', 'point', 'kill', '7'], ['american health care act', 'family click play', 'want', 'mean', 'know', 'find'], ['reduce carbon dioxide emissions promising new technologies like advanced nuclear power carbon capture', 'lower emissions without crashing', 'carbon utilization hold', 'significant emissions reductions', 'promote innovation', 'key', 'instead', 'economy'], ['potus executive actions', 'leading senate_go ps', 'amicus brief', 'support', 'scotus', 'immigration', 'filing', 'challenge']]
[['afford partisan politics stay strong lisa', 'lgbtq people alaska natives', 'health care access', 'health rights', 'rights', 'voting', 'us', 'thank', 'standing', 'sen', 'protect', 'lives', 'lisamurkowski', 'freedoms'

# Finding Tweets of a Similar Topic

RAKE implementation from from https://github.com/csurfer/rake-nltk

In [0]:
def filterTweetsByTopic(keyWordsList, baseKeywords, party, simCount):
  sameTopicTweets = []
  
  for i, tweetKeywords in enumerate(baseKeywords):
    numSimilar = 0
    for keyword in keyWordsList:
      if keyword in tweetKeywords: 
        numSimilar += 1
      if numSimilar == simCount: 
        if party == 0: 
          sameTopicTweets.append((originalRep[i], i))
        else:
          sameTopicTweets.append((originalDem[i], i))
        break
 
  return sameTopicTweets

In [0]:
SIMILARITY_COUNT = 5
MIN_TWEETS_TO_COMPARE = 10

def getSameTopicTweets(inputSent, party):
  sameTopicTweets = []
  simCount = SIMILARITY_COUNT
 
  rInput = Rake()
  rInput.extract_keywords_from_text(inputSent)
  inputKeywords = rInput.get_ranked_phrases()
  print ('Input keywords: {}'.format(inputKeywords))
  
  simCount = min(SIMILARITY_COUNT, len(inputKeywords))
  
  baseKeywords = allDemKeywords
  if party == 0:
    baseKeywords = allRepKeywords
    
  while (len(sameTopicTweets) < MIN_TWEETS_TO_COMPARE and simCount > 0):
    sameTopicTweets += filterTweetsByTopic(inputKeywords, baseKeywords, party, simCount)
    simCount -= 1
        
  # not enough tweets filtered
  if len(sameTopicTweets) < MIN_TWEETS_TO_COMPARE:
    print ('Preliminary filtering was not sufficient')
    separatedKeywords = []
    for phrase in inputKeywords:
      sep = word_tokenize(phrase)
      separatedKeywords += sep
 
    simCount = min(SIMILARITY_COUNT, len(separatedKeywords))

    while (len(sameTopicTweets) < MIN_TWEETS_TO_COMPARE and simCount > 0):
      sameTopicTweets += filterTweetsByTopic(separatedKeywords, baseKeywords, party, simCount)
      simCount -= 1
  
  if len(sameTopicTweets) == 0: print ('WARNING!! NO OTHER SIMILAR TWEETS BY TOPIC FOUND!!')
  return sameTopicTweets

# Cosine Similarity

Resources: 

*  https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/semantic_similarity_with_tf_hub_universal_encoder.ipynb#scrollTo=Q8F4LNGFqOiq
* https://medium.com/@gaurav5430/universal-sentence-encoding-7d440fd3c7c7


In [0]:
!pip3 install --quiet seaborn

In [0]:
!pip3 install tf-nightly-gpu
config = tf.ConfigProto()
config.graph_options.rewrite_options.shape_optimization = 2
session = tf.Session(config=config)

In [0]:
import matplotlib.pyplot as plt
import os, sys
from sklearn.metrics.pairwise import cosine_similarity
import re
import seaborn as sns

from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"

In [0]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

Preprocessing: getting the sentence embedding vectors of all the Republican and Democratic tweets

In [0]:
def turnSentsToVectors(originalData):
  with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embed(originalData))
    return np.array(message_embeddings)

In [0]:
repSentEmbeddings = turnSentsToVectors(originalRep)
demSentEmbeddings = turnSentsToVectors(originalDem)

Function to find the most similar tweet using cosine similarity.

In [0]:
def getMostSimilarTweet(inputText, party, candidateTweets):
  
  # Get sentence embedding of input
  input_embedding = np.array(turnSentsToVectors([inputText]))
  #print 'Finished input embedding'
  
  def cos_sim(compEmbedding):
    similarity = cosine_similarity(input_embedding, compEmbedding)
    return similarity
  
  originalTweets = originalRep
  oppEmbeddings = repSentEmbeddings
  if party == 1:
    originalTweets = originalDem
    oppEmbeddings = demSentEmbeddings
  
  maxCosSim = -1
  bestOppTweet = None
  for candidateTweet in candidateTweets:
    cosineSim = cos_sim([oppEmbeddings[candidateTweet[1]]])[0][0]
    if maxCosSim < cosineSim: 
      bestOppTweet = candidateTweet[0]
      maxCosSim = cosineSim
  
  return (bestOppTweet, maxCosSim)

In [0]:
def getBleuScore(reference, candidate):
  sepReference = [word_tokenize(reference)]
  sepCandidate = word_tokenize(candidate)
  #cc = SmoothingFunction()
  score = sentence_bleu(sepReference, sepCandidate)
  return score

# **Testing**

Predicting the input as either Democratic (0) or Republican (1).

In [0]:
def getPrediction(in_sentences):
  labels = [0, 1]
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

The function below will classify all tweets and save the results to a csv.

In [0]:
def preClassify():
  numCorrect = 0.0
  testingTweets = []
  for t in tweetsToTest:
    testingTweets.append(t[0])

  predResults = []
  allPred = getPrediction(testingTweets)
  for i, testTweet in enumerate(allPred):
    pred = testTweet[2]
    if pred == tweetsToTest[i][1]:
      numCorrect += 1
    predResults.append((tweetsToTest[i][0], pred))

  predictionResultsDF = pd.DataFrame(predResults, columns = ['text', 'prediction'])
  predictionResultsDF.to_csv('predictionResults.csv')
  !cp predictionResults.csv drive/My\ Drive/

  accuracy = numCorrect/len(tweetsToTest)
  print (accuracy)
  
  return predResults
  

Gets a counter argument for the input using the following algorithm:

1.   Predicts party classification
2.   From the opposite party of the classification, filters the tweets to extract tweets of the same topic as the input
3.   Find most similar tweet from the extracted tweets in step 2 using cosine similarity



In [0]:
def getCounterTweet(inputText, actualParty, predictedParty):
  numPredictedCorrectly = 1
  if actualParty != predictedParty:
    numPredictedCorrectly = 0
    
  sameTopicTweets = getSameTopicTweets(inputText, predictedParty)
  
  outputTweet, cosSimScore = getMostSimilarTweet(inputText, predictedParty, sameTopicTweets)
  
  bScore = getBleuScore(inputText, outputTweet)
 
  return (numPredictedCorrectly, bScore, outputTweet, cosSimScore, predictedParty)
  

Test Cases - From file

Getting all of the test tweets and predictions (pre run) from the csv file into a list.

In [0]:
testTweets = pd.read_csv('cleanedTestTweets.csv', header=None, names=['text', 'classification'])
predictionResults = pd.read_csv('predictionResults.csv', header=None, names=['text', 'prediction'])
uncleanTweetsToTest = extractTweets(testTweets, True)
originalTrain = extractTweets(train, False)
  
print ('Original number of tweets: {}'.format(len(uncleanTweetsToTest)))
tweetsToTest = []
numRemoved = 0
for a in uncleanTweetsToTest:
  if a[0] in originalTrain: 
    numRemoved += 1
    continue
  tweetsToTest.append(a)

print ('Number removed: {}'.format(numRemoved))
print ('New number of tweets: {}'.format(len(tweetsToTest)))
  
allPred = []
for index, row in predictionResults.iterrows():
  allPred.append(row['prediction'])

Original number of tweets: 226
Number removed: 0
New number of tweets: 226


In [0]:
def testFromFile():
  t1 = datetime.datetime.now()

  predictedCorrectly = 0.
  sumBleu = 0.
  sumCos = 0.
  allResults = []
  total = 0.

  #allPredictions = getPredictions(tweetsToTest)
  allPredictions = allPred

  for i, arg in enumerate(tweetsToTest[210:]):
    print (i+210)
    p, bScore, tweet, cosScore, pParty = getCounterTweet(arg[0], arg[1], allPredictions[i+210])
    predictedCorrectly += p
    sumBleu += bScore
    sumCos += cosScore
    tempResult = [arg[0], tweet, bScore, cosScore, pParty, arg[1]]
    allResults.append(tempResult)
    total += 1


  finalResults = pd.DataFrame(allResults, columns = ['Input', 'Counter Tweet', 'Bleu Score', 'Cosine Similarity', 'Predicted Party', 'Actual Party']) 
  finalResults.to_csv('final_results226.csv')
  !cp final_results226.csv drive/My\ Drive/

  print (" ")
  print ("-----FINAL STATS-----")
  print ('Prediction Accuracy = {}'.format(float(predictedCorrectly)/total))
  print ('Bleu Score Average = {}'.format(float(sumBleu)/total))
  print ('Cosine Similarity Average = {}'.format(float(sumCos)/total))

  print (" ")
  t2 = datetime.datetime.now()
  print('--- Total time: {} ---'.format(t2 - t1))


In [0]:
testFromFile()

Test Cases - individual input

In [0]:
#@title Get a counterargument
opinion = 'tax reform is about better-paying jobs and higher wages for the american people sen john thune' #@param {type:"string"}
politicalParty = "Republican" #@param ["Democrat", "Republican"]

if politicalParty == "Democrat":
  politicalParty = 0
else:
  politicalParty = 1

In [0]:
def indivdualTest():
  pred = getPredictions([opinion, 'x'])[0][2]
  p, bScore, tweet, cosScore, pParty = getCounterTweet(opinion, politicalParty, pred)
  print ('Input tweet: {}'.format(opinion))
  print ('Counter tweet: {}'.format(tweet))
  print(" ")
  print ('Predicted Party: {}'.format(pred))
  print ('Actual Party: {}'.format(politicalParty))
  print ('Bleu Score = {}'.format(bScore))
  print ('Cosine Similarity = {}'.format(cosScore))

In [0]:
inidividualTest()

# Error Analysis

Test to see if BERT is classfying by topic. 

In [0]:
def classificationByTopic(inputText, predictedParty):
  oppParty = 0
  if predictedParty == 0:
    oppParty = 1
    
  sameTopicTweetsOpp = getSameTopicTweets(inputText, predictedParty)
  sameTopicTweets = getSameTopicTweets(inputText, oppParty)
  allSame = []
  for t in sameTopicTweetsOpp:
    if t[0] not in allSame:
      allSame.append(t[0])
  for t in sameTopicTweets:
    if t[0] not in allSame:
      allSame.append(t[0])
  topicPredictions = getPrediction(allSame)
  
  print ("Input: {}".format(inputText))
  
  numCorrect = 0.0
  numDem = 0.0
  numRep = 0.0
  for i, t in enumerate(topicPredictions):
    actualParty = predictedParty
    if i < len(sameTopicTweetsOpp):
      actualParty = oppParty
      
    print ("Tweet: {}".format(t[0]))
    print ("Acutal Party: {}".format(actualParty))
    print ("PredictedParty: {}".format(t[2]))
    print (" ")
    if actualParty == t[2]:
      numCorrect += 1
    if actualParty == 0:
      numDem += 1
    else:
      numRep += 1
  
  print ("Accuracy = {}".format(numCorrect/(len(allSame))))
  print ("Num Democratic = {}".format(numDem))
  print ("Num Republican = {}".format(numRep))

In [67]:
classificationByTopic('abortion should be legal', 0)

Input keywords: ['legal', 'abortion']
Input keywords: ['legal', 'abortion']
INFO:tensorflow:Writing example 0 of 491


I0608 22:11:53.890303 140482351376256 run_classifier.py:774] Writing example 0 of 491


INFO:tensorflow:*** Example ***


I0608 22:11:53.893969 140482351376256 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0608 22:12:15.751514 140482351376256 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0608 22:12:15.842892 140482351376256 session_manager.py:493] Done running local_init_op.


Input: abortion should be legal
Tweet: it is absolutely appropriate that the new title regulations take into account the difference between abortion and healthcare abortion is neither healthcare nor family planning which is why the title program has no business funding it jeannemfl 1/4
Acutal Party: 1
PredictedParty: 1
 
Tweet: abortion is not empowering abortion is not healthcare abortion should never be taxpayer funded defund pp
Acutal Party: 1
PredictedParty: 1
 
Tweet: planned parenthood says only of their services are abortion that is a lie time to defundplannedparenthood
Acutal Party: 1
PredictedParty: 1
 
Tweet: our core mission is providing protecting and expanding access to abortion and reproductive health care defund planned parenthood
Acutal Party: 1
PredictedParty: 1
 
Tweet: six presidential candidates voted in favor of murdering babies who survive abortion i do not believe this to be a partisan issue i believe every day americans would see a baby surviving an abortion as 

In [68]:
classificationByTopic('build a wall to stop illegal immigrants from entering the country', 1)

Input keywords: ['stop illegal immigrants', 'wall', 'entering', 'country', 'build']
Input keywords: ['stop illegal immigrants', 'wall', 'entering', 'country', 'build']
INFO:tensorflow:Writing example 0 of 143


I0608 22:12:21.262622 140482351376256 run_classifier.py:774] Writing example 0 of 143


INFO:tensorflow:*** Example ***


I0608 22:12:21.265546 140482351376256 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0608 22:12:39.016431 140482351376256 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0608 22:12:39.095588 140482351376256 session_manager.py:493] Done running local_init_op.


Input: build a wall to stop illegal immigrants from entering the country
Tweet: it will not work it is too expensive it will ruin private property all the money could be better used on health care education or infrastructure how many more reasons not to build the wall do you need seems pretty clear
Acutal Party: 0
PredictedParty: 0
 
Tweet: we need to remember what it feels like to be brave brave does not pit people against each other or put money over lives it does not spread hate cloud truth or build a wall that is what fear does i am running because we are better than that let us be brave together
Acutal Party: 0
PredictedParty: 0
 
Tweet: today heartbreaking attacks in sri lanka come as the country has worked hard to build a common future after years of war our thoughts are with the injured and the families of those killed in today easter sunday attacks
Acutal Party: 0
PredictedParty: 0
 
Tweet: we are not going to build a wall and deport million people that is not happening debate

In [69]:
classificationByTopic('pass the green new deal and save the environment', 0)

Input keywords: ['green new deal', 'save', 'pass', 'environment']
Input keywords: ['green new deal', 'save', 'pass', 'environment']
INFO:tensorflow:Writing example 0 of 31


I0608 22:12:41.368619 140482351376256 run_classifier.py:774] Writing example 0 of 31


INFO:tensorflow:*** Example ***


I0608 22:12:41.371834 140482351376256 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0608 22:13:00.481433 140482351376256 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0608 22:13:00.561488 140482351376256 session_manager.py:493] Done running local_init_op.


Input: pass the green new deal and save the environment
Tweet: do we need a green new deal to save the planet quite the opposite forces of economic repression and centralized decision-making lead to less-clean natural environments the environment is better protected under free economies earth day
Acutal Party: 1
PredictedParty: 1
 
Tweet: the green new deal is the only way to save the earth from utter extinction stop standing in the way troglodytes ok democrats let us vote on your non-binding resolution why are you sabotaging our legislation no that is not an exaggeration davidharsanyi
Acutal Party: 1
PredictedParty: 1
 
Tweet: we must find serious solutions to protect the environment and taxpayers the green new deal is a pipe dream that would bankrupt hardworking arizona families i look forward to working on bipartisan solutions that protect our beautiful outdoors without hurting families
Acutal Party: 1
PredictedParty: 1
 
Tweet: if you care about the environment and conservation you